# 获得数据

根据这些年的探索，对于获得数据的一些心得。

## 商业数据

这部分没有尝试，但可能会是最省事，质量最高的方式。

一般来说，数据都比较贵，个人使用者可能很难承担，但也有几个看起来好象还比较平价的。我记得的有：

- tushare：号称是免费的，但需要积分（通过捐款）。感觉价格不贵。
- 预测者网：通过邮件更新数据。好象价格不高，但数据类型有限。

## 平台数据

比如一些量化平台，可以提供一定程度的高质量数据。但缺点是数据一般不能下载，还有各种限制。每家还有不同的平台需要学习。
可能会有一两个好用的，但没有一家家去试了。也许可以再去试试。

## 免费数据

了解过的：
- baostock:一个小但够用的数据。缺点是感觉网站有段时间没有更新，担心持续性问题。
- qstock：一个比较新的平台。github的星数不多，感觉可能有些数据获取接口不太稳定。没有尝试。看起来在绘图方面下了比较多功夫。
- akshare: 免费数据的老大。数据种类很多，甚至可说是太多了。试了一部分，感觉大多数可能并不好用。比如，有下载的限制，或者其他问题。但维护者一直在更新。我试了一下，觉得东方财富的接口好象相对丰富而且稳定。

## 从akshare获得免费数据

- 免费的东西就意味着麻烦，花费时间和精力，从这个角度来说，天下没有免费的东西是对的。
- 下面的例子是为了获得A股的基本信息，但经历了多次尝试，最后得到最理想的结果是：
  - 东方财富网的包括“名称，代码，行业”
  - cninfo信息最全，但下了一半被限制了。
  - 申万的行业很详细，但覆盖率较差。

In [45]:
import pandas as pd
import akshare as ak
import time

In [147]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    
    return (col_profile)

In [158]:
# em股票代码和名称，从当时报价中提取的。
# 据观察是包含了退市的股票代码，但不知道是不是很全。
# 还包括了已上市但还没有开始交易的股票。
# 没有分板块、分交易所的信息。如果需要分析这个，可以考虑分别下载三个交易所的当日报价，然后提取。
df = ak.stock_zh_a_spot_em()
df = df.iloc[:,1:3]
print(df.shape)
showProfile(df)

(5400, 2)


,列名,类型,缺失比%,样例
0,代码,object,0.0,600525
1,名称,object,0.0,长园集团


In [194]:
# cninfo股票基本情况，只下载了部分。被限制。
# cninfo的基本情况比较全，不象em的有交信息只是代码和名称
# 下了一个小时之后，数据连接被中断了，无法再连接。以后再试试。只是第一次下的比较多，以后可以只下增量，减少下载量。
def get_cninfo():
    code = ak.stock_zh_a_spot_em().iloc[:,1:3].head()
    df = pd.DataFrame()
    for a, b in code.iterrows():
        print(a)
        df = pd.concat([df,ak.stock_profile_cninfo(symbol=b[0])], ignore_index=True)
        time.sleep(1)
    return df

In [150]:
cninfodf = pd.read_pickle('../../cninfo_part1.pkl')
showProfile(cninfodf)

,列名,类型,缺失比%,样例
0,公司名称,object,0.000000,西藏卫信康医药股份有限公司
1,英文名称,object,0.000000,"Tibet Weixinkang Medicine Co., Ltd."
2,曾用简称,object,78.484632,None
3,A股代码,object,0.000000,603676
4,A股简称,object,0.000000,卫信康
5,B股代码,object,99.213724,None
6,B股简称,object,99.213724,None
7,H股代码,object,99.177984,None
8,H股简称,object,99.177984,None
9,入选指数,object,10.864904,国证Ａ指


In [151]:
# legulegu的数据，不包括没有上市和已经退市的。em将上市的也有代码和名称。
# legulegu有很多其他的关键指标的数据，以后研究相关的情况可以下它的数据，或者直接上它网站。
# 目前看还比较稳定。
stock_a_indicator_lg_df = ak.stock_a_indicator_lg(symbol="all")
print(stock_a_indicator_lg_df.shape)
showProfile(stock_a_indicator_lg_df)


(5190, 2)


,列名,类型,缺失比%,样例
0,code,object,0.0,600812
1,stock_name,object,0.0,华北制药


In [152]:
# 申万行业代码
# 申万三级行业组成，2023年5月，只有4213家。
def get_sw_indu():
    sw_index_third_info_df = ak.sw_index_third_info()
    swcode = sw_index_third_info_df.iloc[:,0]
    swdf = pd.DataFrame()
    for a, b in swcode.items():
        swdf = pd.concat([swdf, ak.sw_index_third_cons(symbol=b)])
        print(a)
    return swdf

In [153]:
# em行业数据
# 和em的a股当日交易中提取代码对比，少200多家，大致是已批准但没开始交易的股票，和已经退市的股票。
# 同时，行业数据中包含了em当日交易中没有的90多家，大致是B股的数据。
# 总结来说，行业数据包括了B股，但不包括已经退市和还没有上市的。
def get_em_indu():
    em_code = ak.stock_board_industry_name_em()
    em_df = pd.DataFrame()
    for a, b in em_code.iloc[:,1].items():
        data = ak.stock_board_industry_cons_em(symbol=b)
        data['indu'] = b
        em_df = pd.concat([em_df, data], ignore_index=True)
        time.sleep(1)
        print(a)
    em_indu = em_df.iloc[:,[1,2,-1]]
    em_indu.to_pickle('../../em_code_indu')
    return em_indu

In [157]:
em_indu=pd.read_pickle('../../em_code_indu')
print(em_indu.shape)
showProfile(em_indu)

(5280, 3)


,列名,类型,缺失比%,样例
0,代码,object,0.0,002702
1,名称,object,0.0,海欣食品
2,indu,object,0.0,食品饮料


In [184]:
def get_em_all():

    sh_a_spot_em_df = ak.stock_sh_a_spot_em().iloc[:,[1,2]]
    sh_a_spot_em_df['ex'] = '上海'
    time.sleep(3)
    print('---')
    sz_a_spot_em_df = ak.stock_sz_a_spot_em().iloc[:,[1,2]]
    sz_a_spot_em_df['ex'] = '深圳'
    time.sleep(3)
    print('---')
    bj_a_spot_em_df = ak.stock_bj_a_spot_em().iloc[:,[1,2]]
    bj_a_spot_em_df['ex'] = '北京'
    time.sleep(3)
    print('---')
    cy_a_spot_em_df = ak.stock_cy_a_spot_em().iloc[:,[1,2]]
    cy_a_spot_em_df['market'] = '创业板'
    time.sleep(3)
    print('---')
    kc_a_spot_em_df = ak.stock_kc_a_spot_em().iloc[:,[1,2]]
    kc_a_spot_em_df['market'] = '科创板'
    time.sleep(3)
    print('---')

    all_df = pd.concat([sh_a_spot_em_df,sz_a_spot_em_df,bj_a_spot_em_df], ignore_index=True)
    m_df = pd.concat([kc_a_spot_em_df,cy_a_spot_em_df], ignore_index=True)

    all_df = all_df.merge(m_df.iloc[:,[0,2]], how='left',on='代码')

    all_df[all_df['ex'] == '北京'] = all_df[all_df['ex'] == '北京'].fillna('北交所')
    all_df = all_df.fillna('主板')

    return all_df

In [ ]:
all_df = get_em_all()

In [196]:
showProfile(all_df)

,列名,类型,缺失比%,样例
0,代码,object,0.0,001209
1,名称,object,0.0,洪兴股份
2,ex,object,0.0,深圳
3,market,object,0.0,主板


以下是截至2023年5月，可用的数据。以后更新可以直接使用。

In [197]:
all_df.to_pickle('../../em_code_ex_market.pkl')